# DMS and Chlorophyll-A for CMAQ

---
    author: Barron H. Henderson
    contributors: Brett Gantt, Jeff Willison, Golam Sarwar, and Sara Farrell
    date: 2021-03-23
    last updated: 2024-06-12
---

This notebook creates CMAQ-Ready input files necessary for CMAQ halogen and DMS. DMS chemistry requires DMS concentrations and halogen chemistry relies on chlorophyll concentrations. The Chlorophyll is extracted from NASA MODIS-Aqua level-3 data prodcuts. The DMS is created from monthly climatologies from the Surface Ocean and Lower Atmosphere (SOLAS) project.



# Specify User Input Options

* User input options are described below.
* Most users will update `dom`, `ocnintmpl`, `ocnouttmpl`, and `gdpath`

In [ ]:
# dom : str
#     Name of output domain. For example, 12US1, 36US3, 108NHEMI2. This is used
#     to name outputs and inputs.
dom = '12US1'

# ocnintmpl : str
#     Path to OCEAN file with OPEN (0-1) and SURF (0-1) variables. The path can
#     use strftime templates to construct date specific paths. The strftime
#     function described at https://strftime.net/. For 2016-01-01, here are a
#     few examples: %F = 2016-01-01, %Y%j = 2016001, %m%d = 0101, %b = Jan,
#     %^b = JAN. e.g., OCEAN_%Y%b.nc = OCEAN_2016Jan.nc
ocnintmpl = f'/work/MOD3DATA/2016_{dom}/surface/{dom}_surf.ncf'

# ocnouttmpl : str
#     strftime template to create a new file. The new file will have ocnintmpl
#     variables in addition to DMS and CHLO.
ocnouttmpl = f'output/{dom}/OCEAN_%m_L3m_MC_CHL_chlor_a_{dom}.nc'

# gdpath : str
#    Path to an IOAPI file using the domain (dom). Most of the time you can use
#    your ocean file. If your ocnintmpl is time varying (i.e.,  uses strftime),
#    then you will need to update this to hard code a specific path. e.g.,
#    gdpath = ocnintmpl.replace('%b', 'Jan')
gdpath = ocnintmpl

# overwrite : bool
#     Default False, keep existing intermediate files. This is faster by a lot
#     when redoing a domain, but uses cached results. If True, recreate all.
overwrite = False

# getlatestchlo : bool
#     Default True, discover latest climatology urls from NASA server If False,
#     use a prexisting list of known urls. Known urls can be any url that NetCDF
#     can read (e.g, OpenDAP or local paths). They can also be month specific
#     instead of climatology
getlatestchlo = False

# Install Prerequisites

* On Google Colab or other web-based platforms, you may need to install some non-standard libraries
* To do this, the notebook will use a combination of `apt-get` or `miniconda`, and  `pip`
* Both can be run from within this notebook by specifying options below.

In [ ]:
installprereq = True
installcdo = False
# Use a preinstalled version of cdo in the user path
cdopath = 'cdo'
# Or specify a specific path. For example, at EPA use :
# cdopath = '/work/ROMO/anaconda_envs/cdo-1.9.8/bin/cdo'
# If you intall cdo with miniconda
# cdopath = './miniconda/bin/cdo'

## Install Climate Data Operators

* If you do not have climate data operators, this can install them for you.
    * There are two options, the first is `apt-get install cdo` which works on many Debian based linux systems.
    * The second is more robust and installs from Anaconda.
    * You can also install them yourself.
* This may take a couple minutes

In [ ]:
import os
if installcdo:
    cdopath = './miniconda/bin/cdo'
    if not os.path.exists(cdopath):
        !wget -N -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
        !bash ./Miniconda3-latest-Linux-x86_64.sh  -b -p ./miniconda &> log.miniconda
        !./miniconda/bin/conda install -c conda-forge -q -y 'cdo~=1.9' &> log.cdo

## pip

* Install required prerequsites
* This will be faster than installing cdo.

In [ ]:
if installprereq:
    # If on Google Colab:
    # 1. Copy the requirements file to this folder,
    # 2. remove --user
    !python -m pip install -q --user -r requirements.txt

## Now restart the Runtime

* Optional.
* Click on the Runtime menu.
* Click `Restart Runtime`

# Quick Run

* From here, you can click Runtime, Run after
* This will run everything and then give you a dialog to download the results.

# Import libraries


In [ ]:
%matplotlib inline
importsuccess = False
from urllib.request import urlretrieve
import os
from datetime import datetime
from glob import glob
import zipfile
import warnings

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import cdo
import pycno
import PseudoNetCDF as pnc

os.environ['IOAPI_ISPH'] = '6370000.'
warnings.simplefilter('ignore')
importsuccess = True

## Prepare Climate Data Operators

* Used for spatial interpolation
* Instantiate an operator object (`cdoo`) that will be used in the rest of the project.
* Set `debug=True` for detailed feedback

In [ ]:
%pdb off
cdoo = cdo.Cdo(cdopath)
cdoo.setCdo(cdopath)
print('CDI version', cdoo.version())

# Define the CMAQ Grid

Script will create folders for dom, open file and create a CDO grid mapping definition.

In [ ]:
os.makedirs('cdogrids', exist_ok=True)
os.makedirs(f'output/{dom}', exist_ok=True)
os.makedirs(f'chlor_a/{dom}', exist_ok=True)
os.makedirs(f'dmsclimatology/{dom}', exist_ok=True)

In [ ]:
gdf = pnc.pncopen(gdpath, format='ioapi')
proj = gdf.getproj()
gproj = gdf.getproj(withgrid=True)
crs = proj.crs.to_cf()

In [ ]:
cdogrid = f'cdogrids/{dom}.cdo'
with open(cdogrid, 'w') as cdogf:
    cdogf.write(f"""
gridtype = projection
gridsize = {gdf.NROWS * gdf.NCOLS}
xname = COL
yname = ROW
xsize = {gdf.NCOLS}
ysize = {gdf.NROWS}
xinc = {gdf.XCELL}
xfirst = {gdf.XCELL / 2:.0f}
yinc = {gdf.YCELL}
yfirst = {gdf.YCELL / 2:.0f}
grid_mapping_name = {crs['grid_mapping_name']}
longitude_of_projection_origin = {gdf.XCENT}
latitude_of_projection_origin = {gdf.YCENT}
""")
    for k, v in crs.items():
        cdogf.write(f"{k} = {v}\n".replace('(', '').replace(')', ''))

# DMS Processing

* Surface Ocean Lower Atmospheric Study (SOLAS)
* Created a climatology of DMS (under short-lived species)
* https://www.bodc.ac.uk/solas_integration/


* Steps:
  1. Download the data
  2. Extract the CSV files
  3. Create a netCDF file with known longitude and latitude.
  4. Visualize
  5. Regrid DMS

## Download DMS Climatology

* Downloads file to dmsclimatology folder.
* If you prefer, download it there yourself manually.

In [ ]:
dmsurl = 'https://www.bodc.ac.uk/solas_integration/implementation_products/group1/dms/documents/dmsclimatology.zip'
dmsdest = 'dmsclimatology/dmsclimatology.zip'
if not os.path.exists(dmsdest):
    urlretrieve(dmsurl, dmsdest, reporthook=lambda c, s, t: print(f'\r {min(1, c*s/t):5.1%}', end='', flush=True))

## Extract CSV

In [ ]:
zf = zipfile.ZipFile(dmsdest)
zf.extractall(path='dmsclimatology')

## Create a NetCDF

In [ ]:
dmsncpath = 'dmsclimatology/dmsconcentration.nc'
if overwrite or not os.path.exists(dmsncpath):
    if os.path.exists(dmsncpath):
        os.remove(dmsncpath)

    dmsfile = pnc.PseudoNetCDFFile()
    dmsfile.createDimension('time', 12)
    dmsfile.createDimension('latitude', 180)
    dmsfile.createDimension('longitude', 360)

    timev = dmsfile.createVariable('time', 'd', ('time',))
    refdate = datetime(2000, 1, 1)
    middate = datetime(2000, 1, 15)
    timev[:] = [(middate.replace(month=i) - refdate).total_seconds() / 3600 / 24 for i in range(1, 13)]
    timev.units = 'days since 2000-01-01'
    timev.long_name = 'time'

    lonv = dmsfile.createVariable('longitude', 'd', ('longitude',))
    lonv[:] = np.linspace(-179.5, 179.5, 360)
    lonv.units = 'degrees_east'
    lonv.long_name = 'longitude'

    latv = dmsfile.createVariable('latitude', 'd', ('latitude',))
    latv[:] = np.linspace(-89.5, 89.5, 180)
    latv.units = 'degrees_north'
    latv.long_name = 'latitude'

    dmsfile.setCoords(['time', 'longitude', 'latitude'])

    dmsv = dmsfile.createVariable('DMS', 'f', ('time', 'latitude', 'longitude'), fill_value=-999)
    dmsv.units = 'nM'.ljust(16)
    dmsv.long_name = 'DMS'.ljust(16)
    dmsv.description = "Hansell et al. seawater DMS climatology".ljust(80)

    for ti, monthname in enumerate(['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']):
        dmsdata = pd.read_csv(f'dmsclimatology/DMSclim_{monthname}.csv', names=np.linspace(-179.5, 179.5, 360), na_values=['NaN'])
        dmsdata.set_index(np.linspace(-89.5, 89.5, 180)[::-1], inplace=True)
        dmsv[ti] = dmsdata.values[::-1, ]
        dmsv[np.isnan(dmsv[:])] = np.ma.masked

    dmsfile.save(dmsncpath, format='NETCDF4_CLASSIC', verbose=0).close()
else:
    print('Keeping file', dmsncpath)

## Now regrid DMS to model domain

* `overwrite` is set to False, so if you re-run, it will keep old outputs. 
* `usesetmisstonn` is set to true, this interpolates valid values where data is missing.
  * This is super helpful for missing data.
  * This creates DMS overland.
  * If `usesetmisstonn`, you'll want to mask out overland.

In [ ]:
# Choose one or neither
usesetmisstonn = False
usefillmiss = False

dmsoutpath = f'dmsclimatology/{dom}/dmsconcentration.{dom}.nc'
if overwrite or not os.path.exists(dmsoutpath):
    if os.path.exists(dmsoutpath):
        os.remove(dmsoutpath)
    if usesetmisstonn:
        cdoo.setmisstoc(f'0 -remapycon,{cdogrid} -setctomiss,-999. -setmisstonn', input=dmsncpath, output=dmsoutpath, returnCdf=False)
    elif usefillmiss:
        cdoo.setmisstoc(f'0 -remapycon,{cdogrid} -setctomiss,-999. -fillmiss', input=dmsncpath, output=dmsoutpath, returnCdf=False)
    else:
        cdoo.setmisstoc(f'0 -remapycon,{cdogrid} -setctomiss,-999.', input=dmsncpath, output=dmsoutpath, returnCdf=False)
else:
    print('Keeping file', dmsoutpath)

# Process Monthly Chlorophyll-A

* This tutorial uses climatalogical Chlorophyll
* At the download step, you can switch to year-specific by following special instructions


## Download Monthly Chlorophyll-A

* This tutorial uses climatalogical Chlorophyll
* If this is okay for your project, just run the next cell without any edits.
* You can update it to use year-specific values
  * Go to https://oceancolor.gsfc.nasa.gov/l3/
  * Choose Standard Product, MODIS-Aqua, Chlorophyll Concentration, Monthly, 9km
  * Click Extract or Download
  * Choose "Mapped" and Click "Download"
  * Copy the urls from the webpage list and paste over the results below.
  * Replace the cgi url with the opendap url
    * replace "https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/"
    * with "https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/%Y/%m%d/
    * where %Y, %m, and %d are the year, month and day in the file names. (e.g., 20020701)
    * AQUA_MODIS.%Y%m%d_%Y%m%d.L3m.MO.CHL.chlor_a.9km.nc
  * For example, the original URL https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/AQUA_MODIS.20020701_20020731.L3m.MO.CHL.chlor_a.9km.nc becomes https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/0701/AQUA_MODIS.20020701_20020731.L3m.MO.CHL.chlor_a.9km.nc

* You can also use files that you have already downloaded, by setting urls to point to those files.


In [ ]:
if getlatestchlo:
    from urllib.request import urlopen
    import re

    urls = []
    webroot = 'https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/'
    for prefix in ['2002/0701', '2002/0801', '2002/0901', '2002/1001', '2002/1101', '2002/1201', '2003/0101', '2003/0201', '2003/0301', '2003/0401', '2003/0501', '2003/0601']:
        htmlout = urlopen(webroot + prefix)
        htmltxt = htmlout.read().decode()
        mostrecent = sorted(re.compile('(?<=>).+L3m_MC_CHL_chlor_a_9km.nc(?=</)').findall(htmltxt))[-1]
        urls.append(webroot + prefix + '/' + mostrecent)
    
    urls = '\n'.join(urls)
else:
    urls = """
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/0701/AQUA_MODIS.20020701_20020731.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/0801/AQUA_MODIS.20020801_20020831.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/0901/AQUA_MODIS.20020901_20020930.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/1001/AQUA_MODIS.20021001_20021031.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/1101/AQUA_MODIS.20021101_20021130.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2002/1201/AQUA_MODIS.20021201_20021231.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0101/AQUA_MODIS.20030101_20030131.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0201/AQUA_MODIS.20030201_20030228.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0301/AQUA_MODIS.20030301_20030331.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0401/AQUA_MODIS.20030401_20030430.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0501/AQUA_MODIS.20030501_20030531.L3m.MO.CHL.chlor_a.9km.nc
https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/2003/0601/AQUA_MODIS.20030601_20030630.L3m.MO.CHL.chlor_a.9km.nc
"""

In [ ]:
print(urls)

## Now regrid the Chlorophyll-A

* `overwrite` is set to false, so if you re-run, it will keep old outputs. 
* `usefill` is set to true, this interpolates valid values where data is missing.
  * This is super helpful for missing data.
  * This creates Chlorophyll-A overland.
  * If `usefill`, you'll want to mask out overland using an OCEAN file.

In [ ]:
# Filling missing values over land. This can look quite weird when places
# like the Utah Salt Lake create spikes that persist over land.
# When you are going to mask with LAND, this is useful.
#
# * usefill must be  performed globally, because fillmiss is not supported
#   for the projections... the global application makes it unnecessarily slow.
# * usesetmisstonn is perfomed on the regional grid, so it is fast.
usefill = False
useglobalsetmisstonn = False
usesetmisstonn = True
for chlinpath in urls.split():
    tmppath = os.path.join('chlor_a', os.path.basename(chlinpath))
    chloutpath = os.path.join('chlor_a', dom, os.path.basename(chlinpath.replace('9km', dom)))
    print('Input', chlinpath)
    print('Output', chloutpath)

    if overwrite or not os.path.exists(tmppath):
        print('Downloading..', end='.', flush=True)
        if os.path.exists(tmppath):
            os.remove(tmppath)
        if usefill:
            cdoo.fillmiss(' -selvar,chlor_a', input=chlinpath, output=tmppath, returnCdf=False)
        elif useglobalsetmisstonn:
            cdoo.setmisstonn(' -selvar,chlor_a', input=chlinpath, output=tmppath, returnCdf=False)
        else:
            cdoo.selvar('chlor_a', input=chlinpath, output=tmppath, returnCdf=False)
    else:
        print('Keeping existing..', end='.', flush=True)

    if overwrite or not os.path.exists(chloutpath):
        print('Regridding...', flush=True)
        if os.path.exists(chloutpath):
            os.remove(chloutpath)
        if usesetmisstonn:
            cdoo.setmisstonn(f' -remapycon,{cdogrid} -setctomiss,-32767.', input=tmppath, output=chloutpath)
        else:
            cdoo.setmisstoc(f'0 -remapycon,{cdogrid} -setctomiss,-32767.', input=tmppath, output=chloutpath)
    else:
        print('Keeping existing')
print('Done')

# Combine DMS and Chlorophyll-A in a CMAQ-ready File

* OPEN and SURF will be taken from 1 OCEAN file.
* DMS will be taken from 12-monthly representative days.
* CHLO will be taken from 12-monthly representative files.
* Data will be visualized to confirm.


* Steps
    * Combine
    * Visualize

## Combine DMS and Chlorophyll-A

* DMS and CHLO will be added to monthly copies of the OCEAN file.

In [ ]:
dmsf = pnc.pncopen(f'dmsclimatology/{dom}/dmsconcentration.{dom}.nc', format='netcdf')
ocnoutpaths = []
for chloutpath in sorted(glob(f'chlor_a/{dom}/AQUA*{dom}.nc')):
    print(chloutpath)
    mydate = datetime.strptime(os.path.basename(chloutpath)[11:19], '%Y%m%d')
    # seaicepath = mydate.strftime(f'output/{dom}/SEAICE_2020%m01_CLIM.nc')
    ocnoutpath = mydate.strftime(ocnouttmpl)
    ocnoutpaths.append(ocnoutpath)
    ocnpath = mydate.strftime(ocnintmpl)
    ocnf = pnc.pncopen(ocnpath, format='ioapi')
    if not overwrite and os.path.exists(ocnoutpath):
        print('Keeping', ocnoutpath)
        continue
    if os.path.exists(ocnoutpath):
        os.remove(ocnoutpath)

    ti = mydate.month - 1
    print(ti, mydate)
    dmstimef = dmsf.slice(time=ti)
    dmsinv = dmstimef.variables['DMS']
    chlf = pnc.pncopen(chloutpath, format='netcdf')
    chlinv = chlf.variables['chlor_a']
    outf = pnc.cmaqfiles.ioapi_base.from_ncf(ocnf)
    dmsoutv = outf.copyVariable(dmsinv, key='DMS', dtype='f', dimensions=('TSTEP', 'LAY', 'ROW', 'COL'))
    dmsoutv.var_desc = 'DMS'.ljust(80)
    dmsoutv.units = dmsinv.units[:16].ljust(16)
    chloutv = outf.copyVariable(chlinv, key='CHLO', dtype='f', dimensions=('TSTEP', 'LAY', 'ROW', 'COL'))
    chloutv.var_desc = chloutv.long_name[:80].ljust(80)
    chloutv.long_name = 'CHLO'.ljust(16)
    chloutv.units = chlinv.units[:16].ljust(16)

    island = (ocnf.variables['OPEN'][:] + ocnf.variables['SURF']) == 0
    dmsoutv[island] = 0
    chloutv[island] = 0

    # When chlor_a is masked, the values are missing. Usually, this is due to seaice
    # However, seaice may not perfectly match. If remaining missing values are present
    # Set them to zero.
    dmsoutv[np.ma.getmaskarray(chloutv[:])] = 0
    chloutv[np.ma.getmaskarray(chloutv[:])] = 0


    outf.SDATE = 2000001
    outf.TSTEP = 10000
    outf.updatemeta()
    outf.updatetflag(overwrite=True)
    outf.SDATE = -635
    outf.TSTEP = 0
    outf.variables['TFLAG'][:, :, :] = 0
    outf.FILEDESC = outf.FILEDESC.strip() + (
        f"; OCEAN file {ocnpath}\n"
        + f"DMS added from SOLAS project after regridding and gap filling\n({dmsurl})\n"
        + f"CHLO added from MODIS after regridding and gap filling\n{chloutpath}\n"
    )
    outf.save(ocnoutpath, format='NETCDF3_CLASSIC', complevel=1, verbose=0)

## Print CMAQ-ready File Description


In [ ]:
outf = pnc.pncopen(ocnoutpath, format='ioapi')
print(outf.FILEDESC)
print(outf.HISTORY)

# Download CMAQ-Ready Files

If you are on a cloud processing system, you may want to download the files for reuse on another platform.

In [ ]:
with zipfile.ZipFile('downloaddmschlo.zip', 'w') as zf:
    print(ocnpath)
    zf.write(ocnpath)
    for ocnoutpath in ocnoutpaths:
        print(ocnoutpath)
        zf.write(ocnoutpath)